# 智谱大模型

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## 智谱API

### 安装

由于智谱官方的 zhipuai 需要 pydantic v2，与 langchain的某些包（如langserve）不兼容，因此在langchain中不建议使用。<br>
我从 zhipuai 专门修改了一个 zhipuai_pydantic_v1，用来兼容 pydantic v1，所有API与 zhipuai 完全一致。

我们可以使用 zhipuai_pydantic_v1 来做接下来的所有测试。
（如果你执意要用 zhipuai，那么在使用 langchain_chinese 之前记得将其卸载）。

In [ ]:
!pip install langchain_zhipu

### 同步调用（原生API调用）

In [5]:
from langchain_zhipu import ZhipuAI
client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-4", 
    temperature = 0.95,
    messages=[
        {
            "role": "system",
            "content": "你是一个强大的助手，你的名字叫「文成」，不要啰嗦",
        },
        {
            "role": "assistant",
            "content": "我的名字是「文成」，我是一名AI助手，请向我提问。"
        },
        {
            "role": "user",
            "content": "我是小明，你叫什么名字呢？"
        },
    ],
)

print(response)

model='glm-4' created=1710868411 choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='我是「文成」，您的AI助手。有什么可以帮助您的？', role='assistant', tool_calls=None))] request_id='8489588694179864513' id='8489588694179864513' usage=CompletionUsage(prompt_tokens=82, completion_tokens=15, total_tokens=97)


In [17]:
from langchain_zhipu import ChatZhipuAI

llm = ChatZhipuAI()
llm.get_num_tokens("你是？")

3

In [23]:
from langchain_zhipu import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-4", 
    temperature = 0.95,
    messages=[
        {
            "role": "system",
            "content": "你是一个强大的助手，你的名字叫「文成」",
        },
        {
            "role": "assistant",
            "content": "我的名字是「文成」，我是一名AI助手，请向我提问。"
        },
        {
            "role": "user",
            "content": "我是小明，你叫什么名字呢？"
        },
    ],
)

print(response)

model='glm-4' created=1708402885 choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='我是文成，一个基于人工智能技术的助手。很高兴遇见你，小明！如果你有任何问题或需要帮助，请随时告诉我。', role='assistant', tool_calls=None))] request_id='8367747962398260138' id='8367747962398260138' usage=CompletionUsage(prompt_tokens=80, completion_tokens=29, total_tokens=109)


### 事件流（原生API调用）

In [14]:
from langchain_zhipu import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-3-turbo",  # 填写需要调用的模型名称
    messages=[
        {"role": "system", "content": "你是一个乐于解答各种问题的助手，你的任务是为用户提供专业、准确、有见地的建议。"},
        {"role": "user", "content": "我对太阳系的行星非常感兴趣，特别是土星。请提供关于土星的基本信息，包括其大小、组成、环系统和任何独特的天文现象。"},
    ],
    stream=True,
)
for chunk in response:
    # print(chunk)
    print(chunk.choices[0].delta.content, end="|", flush=True)

土|星|是|太阳|系|中的一|颗|气体|巨星|行星|，|位于|太阳|系的|第六|位|，|是|继|木|星|之后|太阳|系|中|最大的|行星|。|土|星|的一些|基本|信息|如下|：|

大小|和|质量|：|
-| |直径|约为|1|2|0|,|5|3|6|公里|，|大约|是|地球|直径|的|9|.|5|倍|。|
-| |质量|约为|5|.|6|8|3| × |1|0|^|2|6|千克|，|大约|是|地球|质量|的|9|5|倍|。|

组成|：|
-| |土|星|主要由|氢|和|氦|组成|，|这两种|元素|占据了|其|大部分|体积|和|质量|。|
-| |内部|结构|由|金属|氢|、|液|态|氢|和|氦|组成|，|外部|是由|氢|和|氦|组成|的大|气|层|。|

环|系统|：|
-| |土|星|最|著名的|特征|之一|是其|壮|观的|环|系统|，|由|冰|颗粒|、|岩石|碎片|和|尘埃|组成|。|
-| |这些|环|围绕|土|星的|赤|道|平面|，|呈现出|一种|明|亮的|环|状|结构|。|
-| |环|中的|颗粒|物质|大小|不一|，|从|微|小的|尘埃|粒子|到|较大的|岩石|块|。|

独特|的天|文|现象|：|
-| |土|星的|北极|呈|正|六|边|形|，|南极|像|台风|眼|，|这是|由|土|星|大气|层|中|的一种|奇特|现象|造成的|。|
-| 这个|六|边|形的|云|带|被称为|“|土|星的|北极|六|边|形|”，|其|边缘|像|巨大的|喷|气|流|一样|高速|旋转|。|

卫星|：|
-| |土|星|已知|有|6|2|颗|卫星|，|其中|一些|非常|著名|，|包括|泰|坦|（|土|卫|六|）、|瑞|亚|（|土|卫|八|）|和|恩|塞|拉|达|斯|（|土|卫|五|）。|
-| |泰|坦|是|土|星|系统中|最大|和|太阳|系|中|第二|大的|卫星|，|直径|约为|5|,|1|5|0|公里|。|

观测|和|探索|：|
-| |土|星|可以通过|肉眼|观测|到|，|但由于|其|距离|太阳|较|远|，|视|星|等|较|暗|。|
-| |借助|望远镜|，|可以|观察|到|土|星的|环|系统和|一些|卫星|。|
-| |土|星|已被|多个|太空|探测器|访问|过|，|其中|最|著名|的是|卡|西|尼|号|探测器|，|它在|2|0|0|4|年|进入|土|星|轨

### 函数回调

In [14]:
from zhipuai import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-3-turbo",
    messages = [
        {
            "role": "user",
            "content": "你能帮我查询2024年1月1日从北京南站到上海的火车票吗？"
        }
    ],
    tools = [
        {
            "type": "function",
            "function": {
                "name": "query_train_info",
                "description": "根据用户提供的信息，查询对应的车次",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "departure": {
                            "type": "string",
                            "description": "出发城市或车站",
                        },
                        "destination": {
                            "type": "string",
                            "description": "目的地城市或车站",
                        },
                        "date": {
                            "type": "string",
                            "description": "要查询的车次日期",
                        },
                    },
                    "required": ["departure", "destination", "date"],
                },
            }
        }
    ],
    tool_choice="auto",
)
print(response.choices[0].message)

content=None role='assistant' tool_calls=[CompletionMessageToolCall(id='call_8367745282338832529', function=Function(arguments='{"date":"2024-01-01","departure":"北京南站","destination":"上海"}', name='query_train_info'), type='function')]


## 作为 RunnableLambda

### 基本功能

In [97]:
from langchain_zhipu import ZhipuAI

client = ZhipuAI()

def zhipu_chat(messages: [str]):
    response = client.chat.completions.create(
        model="glm-3-turbo",
        messages=messages,
    )
    return(response)

In [98]:
prompt = [
    {"role": "user", "content": "讲一个关于程序员的笑话"},
]

In [99]:
zhipu_chat(prompt)

Completion(model='glm-3-turbo', created=1708146156, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。', role='assistant', tool_calls=None))], request_id='8367745522857040954', id='8367745522857040954', usage=CompletionUsage(prompt_tokens=11, completion_tokens=18, total_tokens=29))

### 包装为 Runnable

In [100]:
from langchain_core.runnables import RunnableLambda

my_zhipu_chat = RunnableLambda(zhipu_chat)

In [101]:
my_zhipu_chat.invoke(prompt)

Completion(model='glm-3-turbo', created=1708146224, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。', role='assistant', tool_calls=None))], request_id='8367743907949124838', id='8367743907949124838', usage=CompletionUsage(prompt_tokens=11, completion_tokens=18, total_tokens=29))

## ZhipuAIChatTiny：最小实现

### 定义

In [107]:
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import (
    BaseChatModel,
    generate_from_stream,
)
from langchain_core.messages import AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from typing import Any, Dict, Iterator, List, Optional, cast

In [161]:
class ZhipuAIChatTiny(BaseChatModel):
    """支持最新的智谱API"""

    client: Optional[ZhipuAI] = None
    
    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        try:
            # 声明 ZhipuAI 的客户端
            from zhipuai import ZhipuAI
            self.client = ZhipuAI()
        except ImportError:
            raise RuntimeError(
                "Could not import zhipuai package. "
                "Please install it via 'pip install zhipuai'"
            )

    @property
    def _llm_type(self) -> str:
        """Return the type of chat model."""
        return "zhipuai"

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """使用 ZhiputAI 的同步调用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
        )

        content = response.choices[0].message.content

        return ChatResult(
            generations=[ChatGeneration(message=AIMessage(content=content))]
        )

### 使用 LLM

In [162]:
llm_tiny = ZhipuAIChatTiny()

In [163]:
llm_tiny.invoke("讲一个关于程序员的一句话笑话")

AIMessage(content='为什么程序员总是携带电脑？因为他们不想被人叫做"裸奔者"。')

### 使用 LLM + OutputParser

In [164]:
chain = llm_tiny | StrOutputParser()
chain.invoke("讲一个关于程序员的一句话笑话")

'为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。'

### 使用 Prompt + LLM + OutputParser

In [165]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("讲一个关于{topic}的笑话")
chain = prompt | llm_tiny | StrOutputParser()

chain.invoke({"topic": "程序员"})

'好的，下面是一个关于程序员的笑话：\n\n为什么程序员不喜欢自然界？\n\n因为那里有太多的bugs（虫子/错误）！\n\n希望这个笑话能让您开心一下！'

### 使用流式输出

<div class="alert alert-warning">
    <b>⚠️ 注意：</b><br>
    因为我们没有实现 stream 方法，基类中的实现将自动调用 invoke 方法，然后一次性返回结果。
</div>

In [166]:
for chunk in llm_tiny.stream("讲一个关于程序员的一句话笑话"):
    print(chunk.content, end="_", flush=True)

为什么程序员总是混淆圣诞节和万圣节？因为 Oct 31 等于 Dec 25。_

## ZhipuAIChatStream：支持流

### 定义

In [167]:
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import (
    BaseChatModel,
    generate_from_stream,
)
from langchain_core.messages import AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult

from typing import Any, Dict, Iterator, List, Optional, cast
from langchain_core.pydantic_v1 import BaseModel, Field

In [269]:
class ZhipuAIChatStream(BaseChatModel):
    """支持最新的智谱API"""

    client: Optional[ZhipuAI] = None

    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        self.client = ZhipuAI()

    @property
    def _llm_type(self) -> str:
        """Return the type of chat model."""
        return "zhipuai"

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """使用 ZhiputAI 的同步调用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
            stream=False
        )

        choice = response.choices[0]

        return ChatResult(
            generations=[
                ChatGeneration(
                    message=AIMessage(content=choice.message.content),
            )],
        )

    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        """使用 ZhiputAI 的事件流用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        # 使用流输出
        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
            stream=True
        )

        for chunk in response:
            choice = chunk.choices[0]
            yield ChatGenerationChunk(
                message=AIMessageChunk(content=choice.delta.content),
            )

### 使用 LLM

In [270]:
llm_stream = ZhipuAIChatStream()

In [271]:
llm_stream.invoke("讲一个程序员的一句话笑话")

AIMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。')

### 使用流式输出

In [272]:
for chunk in llm_stream.stream("讲一个关于程序员的一句话笑话"):
    print(chunk.content, end="_", flush=True)

为什么_程序_员_总是_混淆_圣诞_节_和_万_圣_节_？_因为_ Oct_ _3_1_ _等于_ Dec_ _2_5_。__

<div class="alert alert-success">
    <b>🏅️ 成功了！</b><br>
    现在运行上面的代码，应当可以看到流式输出！
</div>

## ChatZhipuAI：在 langchain_chinese 中完整实现

完整的实现较为繁琐，可以在前面实践的基础上补充：

- 支持所有模型参数
- 支持异步方法
- 支持事件流推送
- 支持智谱的Tool回调
- 支持内置的search工具
- 支持内置的检索工具
- 支持图片生成能力
- 支持调用中的异常
- 提供便利的bind_tools方法
- 提供基于Tool调用的Agent
- ...


In [10]:
!poetry add langchain_chinese@latest

The currently activated Python version 3.9.18 is not supported by the project (>=3.10,<3.12).
Trying to find and use a compatible version. 
Using python3 (3.10.0)
Using version ^0.2.13 for langchain-chinese

Updating dependencies
Resolving dependencies... Downloading https://files.pythonhosted.org/packages/37/6d/121efd7382d5b0284239f4ab1fc1590d86d34ed4a4a2fdb13b30ca8e5740/nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl  13% (109.3s)mm9m (39.8s)loading https://files.pythonhosted.org/packages/33/47/fc483df0b7ddeee987b6ff146c879d3556fcea82cc9aa4203d16e5871c62/numpy-1.26.3-cp310-cp310-macosx_10_9_x86_64.whl (2.2s)Resolving dependencies... Downloading https://files.pythonhosted.org/packages/33/47/fc483df0b7ddeee987b6ff146c879d3556fcea82cc9aa4203d16e5871c62/numpy-1.26.3-cp310-cp310-macosx_10_9_x86_64.whl   8% (3.5s)Resolving dependencies... Downloading https://files.pythonhosted.org/packages/33/47/fc483df0b7ddeee987b6ff146c879d3556fcea82cc9aa4203d16e5871c62/numpy-1.26.3-cp310-cp31

<div>
    <b>ChatZhipuAI的完整实现已经发布</b><br>
    我将完整的实现作为 langchain_chinese 包的一部份发布了。
</div>

**项目地址** [https://pypi.org/project/langchain_chinese/](https://pypi.org/project/langchain_chinese/)<br> 
**源代码地址** [https://github.com/arcstep/langchain_chinese](https://github.com/arcstep/langchain_chinese)

你可以通过 pip 安装：

```
pip install -U langchain_chinese
```
或

```
poetry add langchain_chinese@latest
```

In [2]:
from langchain_chinese import ChatZhipuAI

In [3]:
llm = ChatZhipuAI()

In [5]:
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个强力助手。"),
    ("assistant", "我是一名AI助手，请向我提问。"),
    ("user", "{question}")
])

#### 2024年清明假怎么安排的？

In [6]:
llm = ChatZhipuAI(model="glm-4", temperature=0.01)
chain = (prompt | llm | StrOutputParser())
for s in chain.stream({"question": "2024年清明假怎么安排的？"}):
    print(s, end="|", flush=True)

202|4|年的|清明节|放假|安排|如下|：|4|月|4|日至|6|日|放假|调|休|，|共计|3|天|。|具体|来说|，|4|月|7|日|（|星期|日|）|需要|上班|。|清明节|是|中国的|传统|节日|，|也是|重要的|祭祀|节日|之一|，|主要用于|祭|祖|和|扫|墓|，|以|纪念|已|故|的|亲人|。|在这一|天|，|人们|通常会|携带|祭|品|前往|墓地|，|进行|祭|拜|活动|。|同时|，|清明节|也被|称作|踏|青|节|，|在|春|光明|媚|的季节|里|，|许多人|也会|选择|外出|踏|青|，|享受|春天的|自然|景色|。||

In [104]:
llm = ChatZhipuAI(model="glm-4", temperature=0.01).bind(
    tools=[{
        "type": "web_search", 
        "web_search":{
            "enable":True,
            "search_query": "国务院2024年放假安排"
        }
    }])
chain = (prompt | llm | StrOutputParser())
for s in chain.stream({"question": "2024年清明假怎么安排的？"}):
    print(s, end="|", flush=True)

根据|国务院|的|安排|，|202|4|年|清明节|放假|调|休|如下|：|4|月|4|日至|6|日|放假|，|共|3|天|。|4|月|7|日|（|星期|日|）|上班|。|这意味着|清明节|期间|，|大家|可以从|4|月|4|日开始|连续|休息|三天|，|但是|需要在|4|月|7|日|（|周日|）|补|班|。||

In [105]:
llm = ChatZhipuAI(model="glm-4", temperature=0.01).bind(
    tools=[{
        "type": "web_search", 
        "web_search":{
            "enable":False
        }
    }])
chain = (prompt | llm | StrOutputParser())
for s in chain.stream({"question": "2024年清明假怎么安排的？"}):
    print(s, end="|", flush=True)

截至|我的|知识|更新|日期|（|202|3|年|），|我|无法|提供|202|4|年|清明|假的|具体|安排|，|因为这些|信息|通常|会在|当年|年初|由|相关部门|公布|。|清明节|是|中国的|传统|节日|，|也是|公众|假期|之一|，|通常|会有|1|天的|法定|假期|，|但|具体的|放假|安排|可能会|结合|周末|调|休|形成|小|长假|。

为了|获取|202|4|年|清明|假的|准确|安排|，|建议|您|在|接近|那个|时期|时|关注|官方|发布的|节假日|安排|通知|。||

#### 今天星期几？

In [107]:
llm = ChatZhipuAI(model="glm-4", temperature=0.01)
chain = (prompt | llm | StrOutputParser())
for s in chain.stream({"question": "今天星期几？"}):
    print(s, end="|", flush=True)

抱歉|，|作为一个|AI|，|我没有|实|时的|日期|和时间|信息|。|我|建议|您|查看|您的|设备|上的|日|历来|确定|今天是|星期|几|。||

In [84]:
prompt.invoke({"question": "你是谁？"})

ChatPromptValue(messages=[SystemMessage(content='你是一个强力助手。'), AIMessage(content='我是一名AI助手，请向我提问。'), HumanMessage(content='你是谁？')])

In [85]:
chain.invoke({"question": "你是谁？"})

'我是由 OpenAI 开发的一个人工智能助手，旨在帮助用户回答问题、提供信息、解决问题和执行各种任务。我的设计是为了与用户进行自然对话，并在多个领域提供支持。很高兴见到你，ChatGLM！如果有任何问题或需要帮助，请随时告诉我。'

In [341]:
abc = ZhipuAIChat(model="ABC")
abc

ZhipuAIChat(client=<zhipuai._client.ZhipuAI object at 0x1192c81c0>)

In [342]:
abc.model

'glm-3-turbo'

In [ ]:
await chain.ainvoke({"question": "你是谁？"})